# Sentiment analysis with Word2Vec

### Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN

<hr>

▶️ Run this cell and make sure the version of 📚 [Gensim - Word2Vec](https://radimrehurek.com/gensim/auto_examples/index.html) you are using is ≥ 4.0!

In [1]:
!pip freeze | grep gensim

gensim==4.2.0


# The data


❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number. 

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy.

In [24]:
import numpy as np
from tensorflow.keras.layers import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM, Masking
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

2024-02-06 23:46:14.836511: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-06 23:46:15.206399: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 23:46:15.308994: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-06 23:46:15.309011: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

...

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteT97KZD/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteT97KZD/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


2024-02-06 23:46:44.039794: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-02-06 23:46:44.040530: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2024-02-06 23:46:44.040549: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-8OFTMTN): /proc/driver/nvidia/version does not exist
2024-02-06 23:46:44.043017: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In the previous exercise, you trained a Word2vec representation and converted all your training sentences in order to feed them into a RNN, as shown in the first step of this Figure: 

<img src="word2vec_representation.png" width="400px" />



❓ **Question** ❓ Here, let's re-do exactly what you have done in the previous exercise. First, train a word2vec model (with the arguments that you want) on your training sentence. Store it into the `word2vec` variable.

In [4]:
X_train[0]

['this',
 'was',
 'an',
 'absolutely',
 'terrible',
 'movie',
 "don't",
 'be',
 'lured',
 'in',
 'by',
 'christopher',
 'walken',
 'or',
 'michael',
 'ironside',
 'both',
 'are',
 'great',
 'actors',
 'but',
 'this',
 'must',
 'simply',
 'be',
 'their',
 'worst',
 'role',
 'in',
 'history',
 'even',
 'their',
 'great',
 'acting',
 'could',
 'not',
 'redeem',
 'this',
 "movie's",
 'ridiculous',
 'storyline',
 'this',
 'movie',
 'is',
 'an',
 'early',
 'nineties',
 'us',
 'propaganda',
 'piece',
 'the',
 'most',
 'pathetic',
 'scenes',
 'were',
 'those',
 'when',
 'the',
 'columbian',
 'rebels',
 'were',
 'making',
 'their',
 'cases',
 'for',
 'revolutions',
 'maria',
 'conchita',
 'alonso',
 'appeared',
 'phony',
 'and',
 'her',
 'pseudo',
 'love',
 'affair',
 'with',
 'walken',
 'was',
 'nothing',
 'but',
 'a',
 'pathetic',
 'emotional',
 'plug',
 'in',
 'a',
 'movie',
 'that',
 'was',
 'devoid',
 'of',
 'any',
 'real',
 'meaning',
 'i',
 'am',
 'disappointed',
 'that',
 'there',
 'are

In [10]:
trainword= set(word for sentence in X_train for word in sentence)
len(trainword)

30419

In [5]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

In [7]:
wv[0]

array([ 0.71298176, -0.11270226,  0.11465842,  0.26895747, -0.63644576,
       -0.34519345, -0.560219  ,  0.72027063, -1.2502404 , -0.35012   ,
        0.61865264, -0.8400662 ,  0.08026562,  1.13206   , -0.11083344,
       -0.66089195,  1.0582213 ,  0.068037  , -1.0144647 , -1.2386929 ,
       -0.09395623,  0.20074128,  0.44588864,  0.1115215 ,  0.48280904,
       -0.41285142,  0.33878726,  0.80409807, -1.0095054 , -0.04382391,
        0.1467281 ,  0.42087507,  0.35560283, -0.4527945 , -0.79086286,
       -0.07626786,  0.74390775, -0.34857723, -0.33903077, -0.06441586,
       -0.32046264,  0.04060002, -0.9117397 ,  0.11321305, -0.6161908 ,
       -0.1459005 , -0.7863137 ,  0.6714041 , -0.15478732,  0.48418188,
       -1.0959901 , -0.35496393, -0.9883078 ,  0.33831578,  0.61605346,
        0.14925727,  0.09700355,  0.15759403, -0.16069926,  0.4551886 ,
        0.12253223, -0.41929314,  0.16242976,  0.17736816, -0.01381897,
       -0.3206344 , -0.17635633, -0.15181477, -0.656161  , -0.19

In [12]:
word2vec_3 = Word2Vec(sentences=X_train, vector_size=100, min_count=1)
wv3 = word2vec_3.wv
len(wv3.key_to_index)

30419

Let's reuse the functions of the previous exercise to convert your training and test data into something you can feed into a RNN.

❓ **Question** ❓ Read the following function to be sure you understand what is going on, and run it.

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [38]:
len(X_train_pad)

2500

☝️ To be sure that it worked, let's check the following for `X_train_pad` and `X_test_pad`:
- they are numpy arrays
- they are 3-dimensional
- the last dimension is of the size of your word2vec embedding space (you can get it with `word2vec.wv.vector_size`
- the first dimension is of the size of your `X_train` and `X_test`

✅ **Good Practice** ✅ Such tests are quite important! Not only in this exercise, but in real-life applications. It prevents from finding errors too late and from letting them propagate through the entire notebook.

In [17]:
X_train_pad.shape

(2500, 200, 100)

In [16]:
word2vec.wv.vector_size

100

In [14]:
# TEST ME
for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

# Baseline model

It is always good to have a very simple model to test your own model against - to be sure you are doing something better than a very simple algorithm.

❓ **Question** ❓ What is your baseline accuracy? In this case, your baseline can be to predict the label that is the most present in `y_train` (of course, if the dataset is balanced, the baseline accuracy is 1/n where n is the number of classes - 2 here).

In [18]:
np.unique(y_train)

array([0, 1])

In [19]:
baseline_accuracy= 1/2

# The model

❓ **Question** ❓ Write a RNN with the following layers:
- a `Masking` layer
- a `LSTM` with 20 units and `tanh` activation function
- a `Dense` with 10 units
- an output layer that depends on your task

Then, compile your model (we advise you to use the `rmsprop` as the optimizer - at least to begin with)

In [23]:
model= Sequential()
model.add(Masking(mask_value=0.0, input_shape= (200,100)))
model.add(LSTM(20, activation='tanh'))
model.add(layers.Dense(10, activation='linear'))
model.add(layers.Dense(1, 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer= 'rmsprop', metrics='accuracy')

❓ **Question** ❓ Fit the model on your embedded and padded data - do not forget the early stopping criterion.

❗ **Remark** ❗ Your accuracy will greatly depend on your training corpus. Here just make sure that your performance is above the baseline model (which should be the case even if you loaded only 20% of the initial IMDB data).

In [26]:
es= EarlyStopping(patience=4)
model.fit(X_train_pad, y_train, validation_split=0.2, batch_size=50, epochs=100, callbacks=[es])

Epoch 1/100
40/40 [==============================] - 5s 78ms/step - loss: 0.6931 - accuracy: 0.5170 - val_loss: 0.6751 - val_accuracy: 0.5900
Epoch 2/100
40/40 [==============================] - 2s 62ms/step - loss: 0.6783 - accuracy: 0.5765 - val_loss: 0.6607 - val_accuracy: 0.6180
Epoch 3/100
40/40 [==============================] - 2s 61ms/step - loss: 0.6678 - accuracy: 0.5845 - val_loss: 0.6483 - val_accuracy: 0.6200
Epoch 4/100
40/40 [==============================] - 2s 62ms/step - loss: 0.6502 - accuracy: 0.6175 - val_loss: 0.6503 - val_accuracy: 0.6300
Epoch 5/100
40/40 [==============================] - 3s 63ms/step - loss: 0.6332 - accuracy: 0.6400 - val_loss: 0.6679 - val_accuracy: 0.6040
Epoch 6/100
40/40 [==============================] - 2s 62ms/step - loss: 0.6286 - accuracy: 0.6455 - val_loss: 0.6138 - val_accuracy: 0.6740
Epoch 7/100
40/40 [==============================] - 3s 65ms/step - loss: 0.6225 - accuracy: 0.6500 - val_loss: 0.6150 - val_accuracy: 0.6860
Epoch 

❓ **Question** ❓ Evaluate your model on the test set

In [27]:
model.evaluate(X_test_pad, y_test)

79/79 [==============================] - 2s 18ms/step - loss: 0.6461 - accuracy: 0.6208


[0.6460601687431335, 0.6208000183105469]

# Trained Word2Vec - Transfer Learning

Your accuracy, while above the baseline model, might be quite low. There are multiple options to improve it, as data cleaning and improving the quality of the embedding.

We won't dig into data cleaning strategies here. Let's try to improve the quality of our embedding. But instead of just loading a larger corpus, why not benefiting from the embedding that others have learned? Because, the quality of an embedding, i.e. the proximity of the words, can be derived from different tasks. This is exactly what transfer learning is.



❓ **Question** ❓ List all the different models available in the word2vec thanks to this: 

In [28]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


ℹ️ You can also find the list of the models and their size on the [`gensim-data` repository](https://github.com/RaRe-Technologies/gensim-data#models).

❓ **Question** ❓ Load one of the pre-trained word2vec embedding spaces. 

You can do that with `api.load(the-model-of-your-choice)`, and store it in `word2vec_transfer`

<details>
    <summary>💡 Hint</summary>
    
The `glove-wiki-gigaword-50` model is a good candidate to start with as it is smaller (65 MB).

</details>

In [29]:
word2vec_transfer= api.load('glove-wiki-gigaword-50')

[=================---------------------------------] 35.5% 23.4/66.0MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===========================================-------] 86.8% 57.2/66.0MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



❓ **Question** ❓ Check the size of the vocabulary, but also the size of the embedding space.

In [31]:
len(word2vec_transfer.key_to_index)

400000

In [33]:
word2vec_transfer.vector_size

50

❓ Let's embed `X_train` and `X_test`, same as in the first question where we provided the functions to do so! (There is a slight difference in the `embed_sentence_with_TF` function that we will not dig into)

In [36]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

In [43]:
len(X_train_embed_2)

2500

❓ **Question** ❓  Do not forget to pad your results and store it in `X_train_pad_2` and `X_test_pad_2`.

In [47]:
X_train_pad_2= pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2= pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [48]:
X_train_pad_2.shape

(2500, 200, 50)

❓ **Question** ❓ Reinitialize a model and fit it on your new embedded (and padded) data!  Evaluate it on your test set and compare it to your previous accuracy.

❗ **Remark** ❗ The training here could take some time. You can just compute 10 epochs (this is **not** a good practice, it is just not to wait too long) and go to the next exercise while it trains - or take a break, you probably deserve it ;)

In [52]:
model= Sequential()
model.add(Masking(mask_value=0.0, input_shape= (200,50)))
model.add(LSTM(20, activation='tanh'))
model.add(layers.Dense(10, activation='linear'))
model.add(layers.Dense(1, 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer= 'rmsprop', metrics='accuracy')

In [53]:
es= EarlyStopping(patience=4)
model.fit(X_train_pad_2, y_train, validation_split=0.2, batch_size=50, epochs=20, callbacks=[es])

Epoch 1/20
40/40 [==============================] - 5s 83ms/step - loss: 0.6927 - accuracy: 0.5410 - val_loss: 0.6657 - val_accuracy: 0.6120
Epoch 2/20
40/40 [==============================] - 3s 68ms/step - loss: 0.6651 - accuracy: 0.5975 - val_loss: 0.6571 - val_accuracy: 0.6020
Epoch 3/20
40/40 [==============================] - 3s 72ms/step - loss: 0.6412 - accuracy: 0.6265 - val_loss: 0.6497 - val_accuracy: 0.6480
Epoch 4/20
40/40 [==============================] - 4s 104ms/step - loss: 0.6104 - accuracy: 0.6700 - val_loss: 0.5997 - val_accuracy: 0.6900
Epoch 5/20
40/40 [==============================] - 5s 119ms/step - loss: 0.5886 - accuracy: 0.6945 - val_loss: 0.6000 - val_accuracy: 0.6900
Epoch 6/20
40/40 [==============================] - 5s 122ms/step - loss: 0.5766 - accuracy: 0.7090 - val_loss: 0.5751 - val_accuracy: 0.7180
Epoch 7/20
40/40 [==============================] - 5s 123ms/step - loss: 0.5554 - accuracy: 0.7265 - val_loss: 0.5722 - val_accuracy: 0.7220
Epoch 8/2

In [54]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 75.520%


Because your new word2vec has been trained on a large corpus, it has a representation for many many words! Way more than with your small dataset, especially as you discarded words that were not present more than a given number of times in the train set. For that reason, you have way more embedded words in your train and test set, which makes each iteration longer than previously